# git
refer to: https://www.runoob.com/manual/git-guide/  
https://blog.csdn.net/zhouyy858/article/details/102525289

In [ ]:
git config --global user.name "Fangzheng Cheng"
git config --global user.email "qj00182@pegasus.tech"

ssh-keygen -t rsa -C "qj00182@pegasus.tech"
vim ~/.ssh/id_rsa.pub

In [ ]:
# 更新本地仓库至最新改动
git pull

#创建并切换至指定分支
git checkout -b user/chengfangzheng/adjust_waypoint
# 仅切换至指定分支
git checkout user/chengfangzheng/improve_structure_of_RoutingResponse

# 查看更新
git status

# 添加到缓存区
git add modules/map/pnc_map/pnc_map.cc
git add modules/map/pnc_map/pnc_map.h

# 提交到head
git commit -m "[PPC-58][PNC]improve pnc_map"
# 推送到指定分支(远端仓库)
git push origin user/chengfangzheng/user/chengfangzheng/improve_pnc_map

# 临港分支
git clone -b release_lingang_test http://gitlab.pegasus.tech/pegasus/atlas.git

# gflag in routing

## modules/routing/common/routing_gflags.h
<code>DECLARE_bool(enable_heuristic_cost);</code>
## modules/routing/common/routing_gflags.cc
<code>DEFINE_bool(enable_heuristic_cost, false, "enable heuristic cost");</code>
## modules/routing/conf/routing.conf
<code>--enable_heuristic_cost=false</code>
## code
<code>#include "modules/routing/common/routing_gflags.h"
......
if (FLAGS_enable_heuristic_cost) {
  ......
}</code>


# atlas 安装

In [ ]:
http://confluence.pegasus.tech/pages/viewpage.action?spaceKey=PSTS&title=PSTS+Onboard+Guide
根据这个链接整理

## 开发环境配置
refer to: http://confluence.pegasus.tech/display/PSTS/Atlas+common+commands  
http://confluence.pegasus.tech/pages/viewpage.action?pageId=8622043

In [ ]:
# 下载
git clone http://gitlab.pegasus.tech/pegasus/atlas.git

    
cd atlas
# 下载Perception models, Prediction models, HD Maps, Carla libs
./atlas.sh download all
 
# 进入atlas docker
./atlas.sh dev_start
./atlas.sh dev_into
 
# 编译工程
./atlas.sh build

# 清除编译
./atlas.sh clean -a

# 退出docker
exit
 
# 在退出docker后，关闭docker
./atlas.sh dev_stop

## prediction failed to start问题

In [ ]:
# 退出docker
./atlas.sh dev_stop
# 查看docker image
docker image list

<code>REPOSITORY                            TAG       IMAGE ID       CREATED        SIZE
harbor.pegasus.tech/infra/pegaatlas   1.3.8     4fe2fb2df258   2 weeks ago    17.4GB
harbor.pegasus.tech/infra/pegaatlas   1.3.7     dd62004f6fe0   3 weeks ago    17.4GB
harbor.pegasus.tech/infra/pegaatlas   1.3.5     5b98dfdc749b   7 weeks ago    19.1GB
harbor.pegasus.tech/infra/pegaatlas   1.3.4     dc2a8b095b1a   3 months ago   19.1GB</code>

In [ ]:
# 删除旧的docker image
docker image rm harbor.pegasus.tech/infra/pegaatlas:1.3.8

# routing module

## bazel安装
refer to: https://bazel.build/install/ubuntu  

https://bazel.build/

https://docs.bazel.build/versions/4.2.2/tutorial/cpp.html


https://github.com/bazelbuild

https://github.com/orgs/bazelbuild/repositories?type=all

## 模块编译

In [ ]:
bazel build //modules/routing/...

In [ ]:
bazel build //sw_test/autobot.py

## 地图生成工具

In [ ]:
# base_map.bin -> routintg_map.bin & routintg_map.txt
dir_name=modules/map/data/demo_10km
dir_name=modules/map/data/lingang_odd_extend
./scripts/generate_routing_topo_graph.sh --map_dir ${dir_name}

# base_map.bin -> base_map.txt
python modules/tools/meta_hdmap_window/hdmap_window.py -m /atlas/modules/map/data/demo_10km/base_map.bin -s /atlas/modules/map/data/demo_10km/base_map.txt

python modules/tools/meta_hdmap_window/hdmap_window.py -m /atlas/modules/map/data/lingang_odd_extend/base_map.bin -s /atlas/modules/map/data/lingang_odd_extend/base_map.txt


# base_map.txt -> base_map.bin
python modules/tools/meta_hdmap_window/hdmap_window.py -m /atlas/modules/map/data/demo_10km/base_map.txt -s /atlas/modules/map/data/demo_10km/base_map.bin

python modules/tools/meta_hdmap_window/hdmap_window.py -m /atlas/modules/map/data/lingang_odd_extend/base_map.txt -s /atlas/modules/map/data/lingang_odd_extend/base_map.bin


# base_map.txt -> base_map.bin
bazel-bin/modules/map/tools/bin_map_generator --map_dir=${dir_name} --output_dir=${dir_name}


# base_map.bin -> sim_map.bin & sim_map.txt
dir_name=modules/map/data/demo_10km
bazel-bin/modules/map/tools/sim_map_generator --map_dir=${dir_name} --output_dir=${dir_name}

## 地图版本校验
 

In [ ]:
md5sum modules/map/data/demo_10km/base_map.bin

PS: 同一电脑生成的相同文件，MD5码一样；不同电脑生成的相同文件，MD5码不一致

# routing 测试

## autobot
refer to: http://confluence.pegasus.tech/display/PSTS/How+to+Use+Test+Framework+to+Run+PnC+Simulation  

### routing+planning+control仿真测试

In [ ]:
# Inside the docker container where you run the Atlas build.
./bazel-bin/sw_test/autobot

./bazel-bin/sw_test/autobot --no-failure_retry -case pnc_sim_demo_5km_26m

In [ ]:
./bazel-bin/sw_test/autobot -tag baseline # 必验步骤, 须100%通过

In [ ]:
./bazel-bin/sw_test/autobot -case pnc_sim_demo_10km_200

# bazel build //core/meta/tools/websocket:websocket
./bazel-bin/core/meta/tools/websocket/websocket

In [ ]:
./bazel-bin/sw_test/autobot -case pnc_sim_demo_5km_18 --no-use_perfect_control

### routing仿真测试
#### 执行（不支持-case）

In [ ]:
./bazel-bin/sw_test/autobot -t route_sim

#### 配置
如需调整routing模块仿真(route_sim)的配置文件, 其在**<font color="red">/atlas/sw_test/configs/sim/single_sim/route_sim_example.yaml</font>**

<code>
### Launch before test ###
Launch:
  map_config: <font color="red">changlinroad</font> # (or <font color="red">demo_5km</font>)            <font color="red">step 1.选择地图, 存放路径:/atlas/modules/map/data</font>
  vehicle_config: ES6
  check_model: []
  launch_modules: [routing]
  stop_modules_after_test: True
  cpu_stress: 0
  gpu_stress: 0
### Execute Test ###
Execute:
  test_type: sim # replay || sim
  test_program: route_sim
  case_path: "/atlas/simulation/route_simulation/cases/" # 测试用例存放路径
  case_list: [<font color="red">"route_example_1.yaml",                    # step 2.测试用例列表
              "route_example_2.yaml",
              "route_example_3.yaml",
              "route_example_4.yaml",
              "route_example_5.yaml",
              "route_example_6.yaml",
              "route_example_7.yaml",
              "route_example_8.yaml",
              "route_example_9.yaml",
              "route_example_10.yaml",
              "route_example_11.yaml",
              "route_example_12.yaml",
              "route_example_13.yaml",
              "route_example_14.yaml",
              "route_example_15.yaml",
              "route_example_16.yaml",
              "route_example_17.yaml",
              "route_example_18.yaml",
              "route_example_19.yaml",
              "route_example_20.yaml",
              "route_example_21.yaml",
              "route_example_22.yaml",
              "route_example_23.yaml",
              "route_example_24.yaml",
              "route_example_25.yaml",
              "route_example_26.yaml",
              "route_example_27.yaml",</font>
              ]
### Test Evaluator ####
Evaluator:
  module1:
    meta:
      topic_name: "/atlas/routing_response"
      hz:
        wait_time: 10
        expect_value: 1
        max_error: 5
    function:
      data_correct:
        data: object_velocity
        type: less_than_is_correct
        value: 10
### Test results ###
Log:
  path: "/atlas/sw_test/test_results/log"
Bag:
  path: "/atlas/sw_test/test_results/bag"
report:
  path: "/atlas/sw_test/test_results/report"
</code>




If everything is ok, you will see a window as bellow picture, congratulation, you have run pnc sim and visualization successfully.# bin->txt
python modules/tools/meta_hdmap_window/hdmap_window.py -m /atlas/modules/tools/meta_hdmap_window/testdata/base_map.bin -s /atlas/modules/tools/meta_hdmap_window/testdata/base_map.txt

### 仿真输出
log save to /sw_test/test_results/ folder with timestamp name.  
lane_id sequence in routing log 源于 /atlas/modules/routing/core/navigator.cc

In [ ]:
void PrintDebugData(const std::vector<NodeWithRange>& nodes) {
  AINFO << "Route lane id\tis virtual\tstart s\tend s";
  for (const auto& node : nodes) {
    AINFO << node.GetTopoNode()->LaneId() << "\t"
          << node.GetTopoNode()->IsVirtual() << "\t" << node.StartS() << "\t"
          << node.EndS();
  }
}

delete all logs

In [ ]:
rm -rf sw_test/test_results/

## route_generator


### 在docker内，启动routing模块

In [ ]:
./bazel-bin/modules/routing/routing_main modules/routing/dag/routing.dag

## 发送请求前，检查地图
<code>/modules/common/data/global_flagfile.txt</code>  
    
确保最后一个行是waypoint对应的地图

### 在docker内，发送routing请求
waypoint在/simulation/sensors/route_generator.py 中对应地图位置修改


In [ ]:
./bazel-bin/modules/tools/road_test/route_generator --demo_10km
./bazel-bin/modules/tools/road_test/route_generator --lingang_odd

### log save to /data/log

### 在docker内，启动planning模块

In [ ]:
./bazel-bin/modules/routing/routing_main modules/routing/dag/routing.dag

In [ ]:
./bazel-bin/modules/tools/road_test/route_generator --lingang_odd_extend -i to_dishui_lake_B

In [ ]:
./bazel-bin/modules/planning/planning_main modules/planning/dag/planning.dag

In [ ]:
./bazel-bin/core/meta/tools/websocket/websocket

In [ ]:
meta_bag play xxx.bag --topics /atlas/canbus/chassis /atlas/localization/pose /atlas/prediction /atlas/control /atlas/canbus/chassis_detail /atlas/perception /atlas/perception/traffic_light

# kaleido

refer to: https://foxglove.dev/docs/studio  
http://confluence.pegasus.tech/display/PBD/Kaleido+FAQ

## 卸载

In [ ]:
sudo dpkg -r kaleido-studio

In [ ]:
rm -rf ~/kaleido/config/kaleido-topic.json

## 安装

In [ ]:
sudo dpkg -i kaleido-studio-*-linux-amd64.deb

## 启动

In [ ]:
kaleido-studio

# meta_topic

In [ ]:
meta_topic list

In [ ]:
meta_topic echo /atlas/routing_request

In [ ]:
meta_topic echo /atlas/routing_response

# routing地图工具

## scenario_creator 取点工具

scenario_creator.py is a tool for creating PnC simulation cases.

How to generate a waypoints list data for PnC simulation.
1. Run this tool by command "bazel-bin/simulation/tools/scenario_creator --changlinroad", 
(supports changlinroad, caohejing, jiangliulu, demo_5km maps now), then a matplotlib window will popup.
2. You can move your mouse then key "z/Z" to zoom in map, you can key "a/A" to reset window limit.
3. Move your mouse to the point where you wanna to place a ego routing point, then key "e/E", a blue point will be drawed on map window with point id (the first point's id is 1). move your mouse and key agian to generate next point.
4. Move your mouse to the point where you wanna to place a npc control point, then key "n/N", a red point will be drawed on map window with point id (the first point's id is 1). move your mouse and key agian to generate next point.
5. After choose all points you wanna, then key "enter" to generate npc trajectort, please wait a moment, the npc's position, speed and yaw
curve will draw on the right of window.
6. You can key "q" to quit matplotlib window.
7. Switch to your terminal which you run this script, the waypoints will be printed here (utm coordinate)
8. Finally, you can copy this waypoints txt and paste it on yaml file to create a new case.

note: This tool not supports clear point created by mistake, you need restart this tool script. only supports one npc waypoints generate now.
note: Not supports input speed when you add a new point, the default is 5mps, you can change this value on yaml file.

In [ ]:
./bazel-bin/simulation/tools/gui/scenario_creator --changlinroad

In [ ]:
./bazel-bin/simulation/tools/gui/scenario_creator --demo_5km

快捷键|describe|描述
:-|:-|:-
z/Z|zoom in map window|以鼠标点为中心放大
a/A|reset map window|还原
e/E|move your mouse and key "e or E" to place a ego routing request point|取点
n/N|move your mouse and key "n or N" to place a npc trajectory point|暂未使用
enter|generate npc trajectory and print on terminal|在终端打印取点坐标
q/Q|quit matplotlib|关闭


## plot_routing_response

In [ ]:
./bazel-bin/modules/routing/tools/plot_routing_response -p sw_test/test_results/test_data/pnc_sim/pnc_replay_sim_demo_10km_1/log/routing/ -i -1

In [ ]:
./bazel-bin/modules/routing/tools/plot_routing_response -p ./sw_test/test_results/test_data/pnc_sim/pnc_replay_sim_demo_10km_1/log/routing/ -i -1

## check_routing_map

In [ ]:
./bazel-bin/modules/routing/tools/check_routing_map -p ./modules/map/data/lingang_odd/

## meta_hdmap_window 可视化工具
refer to： http://confluence.pegasus.tech/display/PPC/How+to+use+hdmap_window+tool  
https://docs.python.org/zh-cn/3/howto/argparse.html

### demo_5km 应用

In [ ]:
# 显示所有车道(lane)
python modules/tools/meta_hdmap_window/hdmap_window.py -m /atlas/modules/map/data/demo_5km/base_map.bin

In [ ]:
# bin->txt
python modules/tools/meta_hdmap_window/hdmap_window.py -m /atlas/modules/map/data/demo_5km/base_map.bin -s /atlas/modules/map/data/demo_5km/base_map.txt

In [ ]:
# 显示所有车道及其id
python modules/tools/meta_hdmap_window/hdmap_window.py -m /atlas/modules/map/data/demo_5km/base_map.bin -sl

In [ ]:
# 显示部分指定的车道(公交车站附近)
python modules/tools/meta_hdmap_window/hdmap_window.py -m /atlas/modules/map/data/demo_5km/base_map.bin -l 52_0_-2 245_0_-1 246_0_-1 26_0_-3 26_0_-2 235_0_-1 231_0_-1 3_0_-2

In [ ]:
# 显示所有道路(road)
python modules/tools/meta_hdmap_window/roadshow.py -m /atlas/modules/map/data/demo_5km/base_map.bin

## map_xysl 待学习工具
http://confluence.pegasus.tech/display/PPC/map_xysl

# Debug

atlas 中的 Log 已经从 glog 切换到了 alog，以后 glog 将不能使用（master 已变更）。请大家了解：  

1> 日志文件统一放到了 atlas/data/log 目录下（以任何方式启动程序都可生成）；  
2> 日志内容默认不显示到屏幕，如有需要，可以按照如下步骤打开：  
    s1: 打开 atlas/config/alog/{对应模块}.yaml （程序第一次运行后创建，之后一直有效）；  
    s2: "MuteLog" 项改为 false，同时 "EchoToStderr" 项改为 true；  
    s3: 保存退出  

更多内容可参考：http://confluence.pegasus.tech/pages/viewpage.action?spaceKey=PPS&title=ALog

如有问题，咨询潘永飞

In [ ]:
vscode 在extensions中安装 Docker 插件
2. 在左侧菜单中，选择Docker, 从Individual Containers中选择 harbor.pegasus.tech/infra/pegaatlas:1.3.3

In [ ]:
.vscode -> launch.json

"program" 修改为 "bazel-bin/modules/prediction/script/analyze_obstacle",

按F5运行 analyze_obstacle.py

# capnp
http://capnproto.github.io/pycapnp/

In [ ]:
pip install pycapnp -i https://pypi.mirrors.ustc.edu.cn/simple/

# miniosh

In [ ]:
minibus-log, 12345678